In [1]:
#pip install py-irt

In [2]:
import pandas as pd
import numpy as np
import json

In [3]:
!py-irt train --help

Usage: py-irt train [OPTIONS] MODEL_TYPE DATA_PATH
                    OUTPUT_DIR

Arguments:
  MODEL_TYPE  [required]
  DATA_PATH   [required]
  OUTPUT_DIR  [required]

Options:
  --epochs INTEGER
  --priors TEXT
  --dims INTEGER
  --lr FLOAT
  --lr-decay FLOAT
  --device TEXT        [default: cpu]
  --initializers TEXT
  --config-path TEXT
  --dropout FLOAT      [default: 0.5]
  --hidden INTEGER     [default: 100]
  --help               Show this message and exit.


In [4]:
dados = pd.read_csv("/content/final_3.csv", sep=";", decimal=",")

In [5]:
dados

,Chave,sexo,Estado_civil,Status_empl,Licenca,Tipo_Resid,Residencia,Alcoolatra,Droga,Suic_familia,...,Eixo I: Panico sem agorafobia,Eixo I: Fobia especifica,Eixo I: Fobia social,Eixo I: Obsessivo-compulsivo,Eixo I: Estresse pos-traumatico,Eixo I: Ansiedade generalizada,Eixo II: Personalidade paranoica,Eixo II: Transtorno de personalidade,TOC,idade
0,NDAR_INVLE566FK2,M,3.0,NaN,0.0,3.0,1.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,40.0
1,NDAR_INVMA818VXP,F,1.0,3.0,0.0,4.0,3.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,20.0
2,NDAR_INVHY103MHY,F,1.0,2.0,0.0,1.0,2.0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0
3,NDAR_INVEC849VWE,F,1.0,3.0,0.0,1.0,3.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,30.0
4,NDAR_INVHF792VJY,F,4.0,2.0,0.0,1.0,NaN,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,NDAR_INVRV044HN1,M,1.0,3.0,0.0,2.0,2.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,30.0
3949,NDAR_INVDE913FPZ,F,5.0,1.0,0.0,1.0,NaN,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0
3950,NDAR_INVVR200LE7,F,5.0,3.0,0.0,4.0,2.0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,30.0
3951,NDAR_INVBC964BCX,M,3.0,3.0,0.0,1.0,2.0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0


In [6]:
df_suic = dados[['Chave', 'Alcoolatra', 'Droga', 'Suic_familia', 'Dep_familia',
              'Alc_familia', 'Drog_familia', 'Neuro', 'psiquiatrica',
              'Capaz de desfrutar das coisas', 'Impacto de sua familia e amigos', 'Capaz de tomar decisões importantes',
              'Estudante', 'Insonia', 'Ansiedade', 'Perda de insights', 'Apetite', 'Perda de peso',
              'Ansiedade somática', 'Hipocondriase', 'Sentimentos_culpa', 'Suicidio', 'Trabalho e interesses',
              'Energia', 'Lentidao pensamento e fala', 'Agitação', 'Libido', 'TOC','Pontuação total','idade',
              'Deprimido', 'sexo', 'Estado_civil', 'Tipo_Resid','Anos educacao formal']] 

# Features geradas pelo MarkovBlanket
#df_suic = df_suic[['Chave', 'Suicidio', 'Trabalho e interesses', 'Apetite', 'Sentimentos_culpa', 'Perda de insights',
#'Ansiedade somática', 'Ansiedade', 'Perda de peso', 'Lentidao pensamento e fala',
#'Hipocondriase', 'Energia', 'Libido',  'sexo','Pontuação total', 'Deprimido']]

df_suic['sexo'] = np.where(df_suic['sexo'] == 'F', 1, 0)

df_suic = df_suic.dropna().reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
for col in df_suic.columns:
    if col not in (['Chave']):
      df_suic[col] = df_suic[col].astype(int)

In [8]:
df = pd.DataFrame(df_suic[['Chave', 'Suicidio', 'Ansiedade']])

In [9]:
df = df.rename({'Suicidio': 'Suicidio_norm'}, axis='columns')
df.Suicidio_norm = np.where(df.Suicidio_norm<2,0,1)

df = df.rename({'Ansiedade': 'Ansiedade_norm'}, axis='columns')
df.Ansiedade_norm = np.where(df.Ansiedade_norm<2,0,1)

In [10]:
with open("/content/suicidio_jsonlines.jsonlines", "w") as outfile:
    for i, entry in enumerate(df["Suicidio_norm"]):
      json_line = '{"subject_id": "' + str(df['Chave'][i]) + '", "responses": {"Suicidio_norm": ' + str(df['Suicidio_norm'][i]) + '}}'
      outfile.write(json_line)
      outfile.write('\n')

In [11]:
with open("/content/suicidio_ansiedade_jsonlines.jsonlines", "w") as outfile:
    for i, entry in enumerate(df["Suicidio_norm"]):
      json_line = '{"subject_id": "' + str(df['Chave'][i]) + '", "responses": {"Suicidio_norm": ' + str(df['Suicidio_norm'][i]) + ', "Ansiedade_norm": ' + str(df['Ansiedade_norm'][i]) + '}}'
      outfile.write(json_line)
      outfile.write('\n')

In [12]:
!py-irt train 1pl suicidio_jsonlines.jsonlines model_result_sucidio/ --lr 0.1 --epochs 150 2>&1

[15:55:42] config: model_type='1pl' epochs=150 priors=None             ]8;id=1667663742.039334-659722;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:66
           initializers=[] dims=None lr=0.1 lr_decay=0.9999                     
           dropout=0.5 hidden=100 vocab_size=None                               
           data_path: suicidio_jsonlines.jsonlines                     ]8;id=1667663742.040796-558911;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:68
           output directory: model_result_sucidio/                     ]8;id=1667663742.0419507-238870;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:69
False
None
           Training Model...                                           ]8;id=1667663742.1180625-296056;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:73
{'device': 'cpu', 'num_items': 1, 'num_subjects': 3612}
[15:55:42] Parsed Model Args: {'device': 'cpu

In [13]:
!py-irt train 1pl suicidio_ansiedade_jsonlines.jsonlines model_result_sucidio_ansiedade/ --lr 0.1 --epochs 150 2>&1

[15:55:47] config: model_type='1pl' epochs=150 priors=None             ]8;id=1667663747.103286-233620;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:66
           initializers=[] dims=None lr=0.1 lr_decay=0.9999                     
           dropout=0.5 hidden=100 vocab_size=None                               
           data_path: suicidio_ansiedade_jsonlines.jsonlines           ]8;id=1667663747.1056252-741385;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:68
           output directory: model_result_sucidio_ansiedade/           ]8;id=1667663747.1073148-534044;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:69
False
None
           Training Model...                                           ]8;id=1667663747.223572-281026;file:///usr/local/lib/python3.7/dist-packages/py_irt/cli.py\cli.py]8;;\:73
{'device': 'cpu', 'num_items': 2, 'num_subjects': 3612}
[15:55:47] Parsed Model Args: {'device': 'cpu

In [14]:
!cat model_result_sucidio/best_parameters.json

{"ability": [-0.43970659375190735, 0.055632684379816055, 0.3569816052913666, -0.48140543699264526, -0.3516671061515808, -0.5101853013038635, -0.3512064814567566, -0.5814077258110046, -0.0808008685708046, 0.6607082486152649, 0.871628999710083, -0.06970927119255066, 0.253606915473938, -0.43656909465789795, -0.43867602944374084, -0.2893824279308319, -0.31465014815330505, 0.12429123371839523, -0.6256315112113953, -0.21096965670585632, -0.6885170340538025, -0.6453474164009094, 0.7814605832099915, -0.6494342088699341, -0.0678514689207077, 0.747225284576416, 0.7798923850059509, -0.466172993183136, -0.02555854618549347, -0.27774304151535034, -0.22068023681640625, -0.13408347964286804, -0.39976102113723755, 0.4875556230545044, -0.39703860878944397, -0.22842901945114136, -0.05012945085763931, 0.5905977487564087, 0.7423571944236755, 0.2571568787097931, -0.08791207522153854, -0.06345844268798828, -0.2426016628742218, -0.12347019463777542, -0.4799236059188843, -0.36220771074295044, -0.7031005620956

In [15]:
!cat model_result_sucidio_ansiedade/best_parameters.json

{"ability": [0.1424102783203125, 0.22310230135917664, 1.04319167137146, 0.17018373310565948, 0.18010540306568146, 1.0574524402618408, 0.33780771493911743, -0.16314692795276642, 1.3915952444076538, 0.751161515712738, 1.3357635736465454, -1.0353586673736572, 1.0955307483673096, 0.7685150504112244, 0.056002452969551086, 0.5039496421813965, 1.1350170373916626, 0.1285933256149292, 0.06311611086130142, -0.02699003927409649, 0.4812101125717163, -0.0642094612121582, -0.7090050578117371, 0.38773113489151, -0.1075098067522049, -0.12465083599090576, 0.023229293525218964, 1.1824615001678467, -0.6254060864448547, 0.1979512721300125, 0.25143682956695557, 0.8698263764381409, 0.07432197779417038, 0.12523871660232544, -0.008846752345561981, 0.4758489727973938, 0.2243083119392395, 0.2207496166229248, 0.3038971722126007, 0.062075864523649216, 1.1243503093719482, -0.8488025665283203, 1.2033710479736328, 0.9273483157157898, 0.1609436273574829, -0.09563109278678894, 0.12879781424999237, 1.0266963243484497, 

In [16]:
with open("/content/model_result_sucidio/best_parameters.json") as json_file:
  model_result_sucidio = json.load(json_file)

In [17]:
with open("/content/model_result_sucidio_ansiedade/best_parameters.json") as json_file:
  model_result_sucidio_ansiedade = json.load(json_file)

In [18]:
df_result_suicidio = pd.DataFrame(columns=["ability_suicidio", "Chave"])
df_result_suicidio_ansiedade = pd.DataFrame(columns=["ability_suicidio_ansiedade", "Chave"])

In [19]:
df_result_suicidio["ability_suicidio"] = model_result_sucidio["ability"]
df_result_suicidio["Chave"] = model_result_sucidio["subject_ids"].values()

df_result_suicidio_ansiedade["ability_suicidio_ansiedade"] = model_result_sucidio_ansiedade["ability"]
df_result_suicidio_ansiedade["Chave"] = model_result_sucidio_ansiedade["subject_ids"].values()

In [20]:
df_suic = df_suic.merge(df_result_suicidio, how='inner', on="Chave")
df_suic = df_suic.merge(df_result_suicidio_ansiedade, how='inner', on="Chave")

In [21]:
df_suic.to_csv("/content/df_suicidio_ansiedade_ability.csv")

In [22]:
df_suic

,Chave,Alcoolatra,Droga,Suic_familia,Dep_familia,Alc_familia,Drog_familia,Neuro,psiquiatrica,Capaz de desfrutar das coisas,...,TOC,Pontuação total,idade,Deprimido,sexo,Estado_civil,Tipo_Resid,Anos educacao formal,ability_suicidio,ability_suicidio_ansiedade
0,NDAR_INVLE566FK2,0,0,0,1,1,1,1,1,2,...,3,25,40,2,0,3,3,12,0.836013,1.173058
1,NDAR_INVMA818VXP,0,0,0,1,1,0,0,0,1,...,0,24,20,3,1,1,4,18,-0.028962,0.128593
2,NDAR_INVHY103MHY,0,0,1,1,0,1,0,0,2,...,0,19,20,2,1,1,1,12,-0.266992,0.090558
3,NDAR_INVEC849VWE,0,0,0,1,1,0,0,0,1,...,6,33,30,2,1,1,1,14,0.380615,0.847915
4,NDAR_INVHF792VJY,1,0,0,1,1,0,0,1,3,...,0,27,40,3,1,4,1,10,-0.171591,0.121865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3607,NDAR_INVRV044HN1,0,0,0,1,1,1,0,0,2,...,2,17,30,2,0,1,2,16,0.734122,1.089779
3608,NDAR_INVDE913FPZ,0,0,0,1,0,0,1,0,1,...,0,22,50,2,1,5,1,10,0.064498,-0.064209
3609,NDAR_INVVR200LE7,0,0,0,1,0,0,0,0,1,...,1,23,30,3,1,5,4,23,0.392840,1.312218
3610,NDAR_INVBC964BCX,0,0,0,0,0,0,0,0,2,...,0,16,50,2,0,3,1,16,-0.501385,-0.111568
